In [ ]:
import numpy as np

from scipy.integrate import solve_ivp
from scipy.optimize import root

from ipywidgets import interact, FloatSlider, Dropdown

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import row, column
from bokeh.models import PrintfTickFormatter
output_notebook(hide_banner=True)

# Van der Pol equation

We consider the following problem :

$$
\left\{ 
\begin{aligned} 
{\mathrm d}_t x & = \epsilon^2 \left(y-\frac{x^3}{3}+x \right)\\ 
{\mathrm d}_t y & = -x 
\end{aligned} 
\right. 
\qquad{} \text{with } \epsilon > 0
$$

In [ ]:
class vanderpol_model:

    def __init__(self, eps):
        self.eps = eps

    def fcn(self, t, y):
        y1, y2 = y
        eps = self.eps
        y1_dot = (eps**2)*(y2-y1**3/3+y1)
        y2_dot = -y1
        return np.array([y1_dot, y2_dot])

    def jac(self, t, y):
        y1, y2 = y
        eps = self.eps
        return np.array([[eps**2*(-y1**2 + 1), eps**2], [-1, 0]])

## Quasi-exact solution

The quasi-exact solution is obtained by using an explicit Runge-Kutta method of order 8 with stepsize control and fine tolerances due to Dormand and Prince.

In [ ]:
tini = 0. 
tend = 10.

yini = (0.5, 0)

vdpm = vanderpol_model(eps=1)
fcn = vdpm.fcn  

sol = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=1e-09, atol=1e-12)

fig_sol = figure(x_range=(tini, tend), width=800, height=300, title="Solution")
plt_sol = fig_sol.line(sol.t, sol.y[0], legend_label="x", line_width=2)    
fig_sol.legend.location = "top_left"
fig_sol.xaxis.axis_label = "t"
fig_sol.yaxis.axis_label = "x"

x = np.linspace(-3,3,1000)
slow_manifold = x**3/3 - x

fig_pp = figure(x_range=(-3, 3), y_range=(-3, 3), width=400, height=400, title="Phase portrait")
plt_pp = fig_pp.line(x, slow_manifold, legend_label="slow manifold", line_width=2, color='red')   
plt_pp = fig_pp.line(sol.y[0], sol.y[1], line_width=2) 
fig_pp.legend.location = "top_left"
fig_pp.xaxis.axis_label = "x"
fig_pp.yaxis.axis_label = "y"

show(column(fig_sol, fig_pp), notebook_handle=True)

def update(eps) :
    vdpm = vanderpol_model(eps)
    fcn = vdpm.fcn  
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1e-09, atol=1e-12)
    plt_sol.data_source.data = dict(x=sol.t, y=sol.y[0])
    plt_pp.data_source.data = dict(x=sol.y[0], y=sol.y[1])
    push_notebook()

interact(update, eps=FloatSlider(min=1.,max=20.,step=1., value=1., continuous_update=False));

## Characterization of stiffness

In [ ]:
tini = 0. 
tend = 10.

yini = (0.5, 0)

vdpm = vanderpol_model(eps=1)
fcn = vdpm.fcn
jac = vdpm.jac

sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1e-09, atol=1e-12)
    
eig_vals = np.zeros((sol.t.size, 2), dtype=np.complex_)
for it in range(0,sol.t.size):
    eig_vals[it] = np.linalg.eigvals(jac(0., sol.y[:,it]))
lambda1 = eig_vals[:, 0]
lambda2 = eig_vals[:, 1]

fig_real = figure(x_range=(tini, tend), plot_height=300, plot_width=900, 
                  title = "Real part of eigenvalues (click on legend entries to hide corresponding plot)")
fig_imag = figure(x_range=(tini, tend), plot_height=300, plot_width=900, 
                  title = "Imaginary part of eigenvalues (click on legend entries to hide corresponding plot)")

plt_real1 = fig_real.line(sol.t, np.real(lambda1), legend_label="lambda1")
plt_real2 = fig_real.line(sol.t, np.real(lambda2), color="Green", legend_label="lambda2")

plt_imag1 = fig_imag.line(sol.t, np.imag(lambda1), legend_label="lambda1")
plt_imag2 = fig_imag.line(sol.t, np.imag(lambda2), color="Green", legend_label="lambda2")

fig_real.legend.click_policy="hide"
fig_imag.legend.click_policy="hide"

show(column(fig_real, fig_imag), notebook_handle=True)

def update(eps):

    vdpm = vanderpol_model(eps)
    fcn = vdpm.fcn
    jac = vdpm.jac

    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=1e-09, atol=1e-12)

    eig_vals = np.zeros((sol.t.size, 2), dtype=np.complex_)
    for it in range(0,sol.t.size):
        eig_vals[it] = np.linalg.eigvals(jac(0., sol.y[:,it]))
    lambda1 = eig_vals[:, 0]
    lambda2 = eig_vals[:, 1]

    plt_real1.data_source.data = dict(x=sol.t, y=np.real(lambda1))
    plt_real2.data_source.data = dict(x=sol.t, y=np.real(lambda2))
    plt_imag1.data_source.data = dict(x=sol.t, y=np.imag(lambda1))
    plt_imag2.data_source.data = dict(x=sol.t, y=np.imag(lambda2))

    push_notebook()

interact(update, eps=FloatSlider(min=1.,max=20.,step=1., value=1., continuous_update=False));

## Dormand and Price method

### Method of order 5

To compute global error, the quasi-exact solution is obtained by using an explicit Runge-Kutta method of order 8 with stepsize control and fine tolerances due to Dormand and Prince.

In [ ]:
tini = 0. 
tend = 10.

yini = (0.5, 0)

vdpm = vanderpol_model(eps=10)
fcn = vdpm.fcn
jac = vdpm.jac
    
fig_sol = figure(x_range=(tini, tend),  plot_height=300, plot_width=900, title="Solution")
fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=900, title="Global error")
fig_dt = figure(x_range=(tini, tend), plot_height=300, plot_width=900, title="Time step")
fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")

tol = 1.e-4
sol_rk45 = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
sol_exa = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=1e-09, atol=1e-12, t_eval=sol_rk45.t)
err_x = np.abs(sol_exa.y[0] - sol_rk45.y[0])

plt_sol_x_x = fig_sol.x(sol_rk45.t, sol_rk45.y[0], legend_label="x", line_width=2)
plt_sol_l_x = fig_sol.line(sol_rk45.t, sol_rk45.y[0], legend_label="x")    
fig_sol.legend.location = "top_left"
fig_sol.xaxis.axis_label = "t"

plt_err_x = fig_err.x(sol_rk45.t, err_x, line_width=2, legend_label="|x - xexa|")
fig_err.legend.location = "bottom_left"
fig_err.xaxis.axis_label = "t"

dt = sol_rk45.t[1::] - sol_rk45.t[0:-1]
plt_dt = fig_dt.quad(top=sol_rk45.t[1:]-sol_rk45.t[:-1], left=sol_rk45.t[:-1], right=sol_rk45.t[1:], 
                     bottom=np.zeros(sol_rk45.t.size-1), line_color="white", alpha=0.5)
fig_dt.xaxis.axis_label = "t"

show(column(fig_sol, fig_err, fig_dt), notebook_handle=True)

def update(eps, tol):

    vdpm = vanderpol_model(eps)
    fcn = vdpm.fcn
    jac = vdpm.jac

    sol_rk45 = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=1e-09, atol=1e-12, t_eval=sol_rk45.t)
    err_x = np.abs(sol_exa.y[0] - sol_rk45.y[0])
    dt = sol_rk45.t[1::] - sol_rk45.t[0:-1]

    plt_sol_x_x.data_source.data = dict(x=sol_rk45.t, y=sol_rk45.y[0])
    plt_sol_l_x.data_source.data = dict(x=sol_rk45.t, y=sol_rk45.y[0])
    plt_err_x.data_source.data = dict(x=sol_rk45.t, y=err_x)
    plt_dt.data_source.data = dict(top=sol_rk45.t[1:]-sol_rk45.t[:-1], bottom=np.zeros(sol_rk45.t.size-1), 
                                   left=sol_rk45.t[:-1], right=sol_rk45.t[1:])
    print("   Number of time step : " + str(sol_rk45.t.size))
    print("   Number of function evaluations : " + str(sol_rk45.nfev))

    push_notebook()

dtol={'1e-2':1e-2, '1e-4':1e-4, '1e-6':1e-6, '1e-8':1e-8}     
interact(update, eps=FloatSlider(min=1.,max=20.,step=1., value=10., continuous_update=False), 
         tol=Dropdown(options=dtol, value=1.e-4, description='tol'));

### Method of order 8

To compute global error, the quasi-exact solution is obtained by using an explicit Runge-Kutta method of order 8 with stepsize control and fine tolerances due to Dormand and Prince.

In [ ]:
tini = 0. 
tend = 10.

yini = (0.5, 0)

vdpm = vanderpol_model(eps=10)
fcn = vdpm.fcn
jac = vdpm.jac
    
fig_sol = figure(x_range=(tini, tend),  plot_height=300, plot_width=900, title="Solution")
fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=900, title="Global error")
fig_dt = figure(x_range=(tini, tend), plot_height=300, plot_width=900, title="Time step")

tol = 1.e-4
sol_dopri853 = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=tol, atol=tol)
sol_exa = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=1e-09, atol=1e-12, t_eval=sol_dopri853.t)
err_x = np.abs(sol_exa.y[0] - sol_dopri853.y[0])

plt_sol_x_x = fig_sol.x(sol_dopri853.t, sol_dopri853.y[0], legend_label="x", line_width=2)
plt_sol_l_x = fig_sol.line(sol_dopri853.t, sol_dopri853.y[0], legend_label="x")
fig_sol.legend.location = "top_left"
fig_sol.xaxis.axis_label = "t"

plt_err_x = fig_err.x(sol_dopri853.t, err_x, line_width=2, legend_label="x")
fig_err.legend.location = "bottom_left"
fig_err.xaxis.axis_label = "t"

dt = sol_dopri853.t[1::] - sol_dopri853.t[0:-1]
plt_dt = fig_dt.quad(top=sol_dopri853.t[1:]-sol_dopri853.t[:-1], left=sol_dopri853.t[:-1], right=sol_dopri853.t[1:], 
                     bottom=np.zeros(sol_dopri853.t.size-1), line_color="white", alpha=0.5)
fig_dt.xaxis.axis_label = "t"

show(column(fig_sol, fig_err, fig_dt), notebook_handle=True)

def update(eps, tol):

    vdpm = vanderpol_model(eps)
    fcn = vdpm.fcn

    sol_dopri853 = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=tol, atol=tol)
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="DOP853", rtol=1e-09, atol=1e-12, t_eval=sol_dopri853.t)
    err_x = np.abs(sol_exa.y[0] - sol_dopri853.y[0])   
    dt = sol_dopri853.t[1::] - sol_dopri853.t[0:-1]

    plt_sol_x_x.data_source.data = dict(x=sol_dopri853.t, y=sol_dopri853.y[0])
    plt_sol_l_x.data_source.data = dict(x=sol_dopri853.t, y=sol_dopri853.y[0])
    plt_err_x.data_source.data = dict(x=sol_dopri853.t, y=err_x)
    plt_dt.data_source.data = dict(top=sol_dopri853.t[1:]-sol_dopri853.t[:-1], left=sol_dopri853.t[:-1], 
                                   right=sol_dopri853.t[1:], bottom=np.zeros(sol_dopri853.t.size-1))

    print("   Number of time step : " + str(sol_dopri853.t.size))
    print("   Number of function evaluations : " + str(sol_dopri853.nfev))

    push_notebook()

dtol={'1.e-2':1.e-2, '1.e-4':1.e-4, '1.e-6':1.e-6, '1.e-8':1.e-8}     
interact(update, eps=FloatSlider(min=1.,max=20.,step=1., value=10., continuous_update=False), 
         tol=Dropdown(options=dtol, value=1.e-4, description='tol'));